In [28]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [29]:
df = pd.read_csv("./Resources/cardio_train.csv", sep = ";", index_col=0)
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


<h3>1. Preprocessing</h3>

In [30]:
# convert naming of gluc
df.loc[df["gluc"] == 1, "gluc"] = "Glucose Normal"
df.loc[df["gluc"] == 2, "gluc"] = "Glucose Above Normal"
df.loc[df["gluc"] == 3, "gluc"] = "Glucose Well Above Normal"

# convert naming of cholesterol
df.loc[df["cholesterol"] == 1, "cholesterol"] = "Cholesterol Normal"
df.loc[df["cholesterol"] == 2, "cholesterol"] = "Cholesterol Above Normal"
df.loc[df["cholesterol"] == 3, "cholesterol"] = "Cholesterol Well Above Normal"

In [31]:
# Convert categorical n > 2 variable to one hot encoding.
categorical_dummies = pd.get_dummies(df[["cholesterol", "gluc"]], dtype=int)
categorical_dummies.columns = categorical_dummies.columns.str.replace("cholesterol_", "")
categorical_dummies.columns = categorical_dummies.columns.str.replace("gluc_", "")

df = pd.concat([df, categorical_dummies], axis=1)
df.drop(columns=["cholesterol", "gluc"], inplace=True)

df.head()


,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,Cholesterol Above Normal,Cholesterol Normal,Cholesterol Well Above Normal,Glucose Above Normal,Glucose Normal,Glucose Well Above Normal
id,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,0,0,1,0,0,1,0,0,1,0
1,20228,1,156,85.0,140,90,0,0,1,1,0,0,1,0,1,0
2,18857,1,165,64.0,130,70,0,0,0,1,0,0,1,0,1,0
3,17623,2,169,82.0,150,100,0,0,1,1,0,1,0,0,1,0
4,17474,1,156,56.0,100,60,0,0,0,0,0,1,0,0,1,0


In [32]:
#reorder the columns
df = df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco',
       'active', 'Cholesterol Above Normal', 'Cholesterol Normal',
       'Cholesterol Well Above Normal', 'Glucose Above Normal',
       'Glucose Normal', 'Glucose Well Above Normal', 'cardio']]

df.head()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,Cholesterol Above Normal,Cholesterol Normal,Cholesterol Well Above Normal,Glucose Above Normal,Glucose Normal,Glucose Well Above Normal,cardio
id,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,0,0,1,0,1,0,0,1,0,0
1,20228,1,156,85.0,140,90,0,0,1,0,0,1,0,1,0,1
2,18857,1,165,64.0,130,70,0,0,0,0,0,1,0,1,0,1
3,17623,2,169,82.0,150,100,0,0,1,0,1,0,0,1,0,1
4,17474,1,156,56.0,100,60,0,0,0,0,1,0,0,1,0,0


In [33]:
# split the data with train test split
X = df[df.columns[:-1]]
y = df["cardio"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Standardize the data
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

<h3>2. ML Models Initial Attempt</h3>

<h4>a. Logistic Regression with All Features</h4>

In [34]:
# try logistic regression model 
# define the model
clf = LogisticRegression(solver="lbfgs")

# fit the model
clf.fit(X_train_scaled, y_train)
logistic_predictions = clf.predict(X_test_scaled)

# calculate the accuracy score
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_accuracy


0.7158857142857142

<h4> b. Decision Tree with All Features </h4>

In [35]:
# create a decision feree classifier
from sklearn import tree
tree_clf = tree.DecisionTreeClassifier()

# fit the data and make prediction
tree_clf.fit(X_train_scaled, y_train)
dt_predictions = tree_clf.predict(X_test_scaled)

print(f"Decision Tree Accuracy is {accuracy_score(y_test, dt_predictions)}")

Decision Tree Accuracy is 0.6331428571428571


In [36]:
# see feature importance
print(tree_clf.feature_importances_)

[0.32441875 0.01667178 0.14468933 0.15804474 0.23184334 0.0424772
 0.0074589  0.00666206 0.0135507  0.00644428 0.00611565 0.02246513
 0.00724082 0.00634548 0.00557186]


<h4> c. Random Forest with All Features </h4>

In [37]:
# create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200)

# fit the data to the model
rf_clf.fit(X_train_scaled, y_train)

# compute the accuracy
rf_predictions = rf_clf.predict(X_test_scaled)

print(f"Random Forest Accuracy is {accuracy_score(y_test, rf_predictions)}")


Random Forest Accuracy is 0.7158857142857142


In [38]:
# see feature importance
print(rf_clf.feature_importances_)

[0.31060896 0.01211528 0.15860784 0.17707693 0.17644332 0.09102195
 0.00739419 0.00651087 0.0113216  0.00469818 0.01542913 0.01637769
 0.00347095 0.00518745 0.00373567]


<h4> d. Feed Forward Network with All Features</h4>

In [39]:
# MLP Model
# model definition
mlp = tf.keras.models.Sequential()

# adding basic layers
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu", input_dim = 15))
mlp.add(tf.keras.layers.Dense(units = 6, activation = "relu"))
mlp.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
mlp.compile(loss = tf.keras.losses.BinaryCrossentropy, optimizer = optimizer, metrics = ["accuracy"])

# fit the mode
mlp.fit(X_train_scaled, y_train, epochs = 30)


Epoch 1/30


c:\Users\David\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 700us/step - accuracy: 0.6816 - loss: 0.6023
Epoch 2/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.7288 - loss: 0.5605
Epoch 3/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - accuracy: 0.7315 - loss: 0.5465
Epoch 4/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step - accuracy: 0.7336 - loss: 0.5457
Epoch 5/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step - accuracy: 0.7300 - loss: 0.5491
Epoch 6/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.7278 - loss: 0.5494
Epoch 7/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - accuracy: 0.7322 - loss: 0.5460
Epoch 8/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - accuracy: 0.7345 - loss: 0.5430
Epoch 9/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - accuracy: 0.7323 - loss: 0.5457
Epoch 10/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 691us/step - accuracy: 0.7359 - loss: 0.5428
Epoch 11/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - accuracy: 0.7347 - loss: 0.5429
Epoch 12/30
1641/16

In [40]:
# evaluate the model, very bad, overfitted.
print(f"Accuracy: {mlp.evaluate(X_test_scaled, y_test)}")

547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.7338 - loss: 0.5415
Accuracy: [0.5494974851608276, 0.7279428839683533]


<h3>3. ML Models With More Relevant Features</h3>

In [41]:
# split the data with train test split
X = df.iloc[:, [0, 2, 3, 4]]
y = df["cardio"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Standardize the data
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

<h4>a. Logistic Regression</h4>

In [42]:
logistic_clf = LogisticRegression(solver="lbfgs")

logistic_clf.fit(X_train_scaled, y_train)
logistic_predictions = logistic_clf.predict(X_test_scaled)

print(f"The accuracy of this logistic regression is {accuracy_score(y_test, logistic_predictions)}")

The accuracy of this logistic regression is 0.7084


<h4> b. Decision Tree </h4>

In [43]:
dt_clf = tree.DecisionTreeClassifier()
dt_clf.fit(X_train_scaled, y_train)
dt_predictions = dt_clf.predict(X_test_scaled)

print(f"The accuracy of this decision tree is {accuracy_score(y_test, dt_predictions)}")

The accuracy of this decision tree is 0.6172


<h4>c. Random Forest Classifier</h4>

In [44]:
# create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200)

# fit the data to the model
rf_clf.fit(X_train_scaled, y_train)

# compute the accuracy
rf_predictions = rf_clf.predict(X_test_scaled)

print(f"Random Forest Accuracy is {accuracy_score(y_test, rf_predictions)}")

Random Forest Accuracy is 0.688


<h4> d. Feed Forward Network </h4>

In [45]:
# MLP Model
# model definition
mlp = tf.keras.models.Sequential()

# adding basic layers and drop out
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu", input_dim = 4))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu"))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu"))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu"))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu"))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 100, activation = "relu"))
mlp.add(tf.keras.layers.Dropout(0.5))
mlp.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
mlp.compile(loss = tf.keras.losses.BinaryCrossentropy, optimizer = optimizer, metrics = ["accuracy"])

# fit the mode
mlp.fit(X_train_scaled, y_train, epochs = 30)

Epoch 1/30


c:\Users\David\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1641/1641 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5064 - loss: 0.6974
Epoch 2/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5504 - loss: 0.6870
Epoch 3/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5955 - loss: 0.6727
Epoch 4/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6160 - loss: 0.6603
Epoch 5/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6372 - loss: 0.6490
Epoch 6/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6567 - loss: 0.6367
Epoch 7/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6708 - loss: 0.6282
Epoch 8/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6901 - loss: 0.6145
Epoch 9/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6948 - loss: 0.6060
Epoch 10/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7013 - loss: 0.6008
Epoch 11/30
1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7059 - loss: 0.5968
Epoch 12/30
1641/1641 ━━━━━━━━━━━━━━━━━━━

In [46]:
# evaluate the model, very bad, overfitted.
print(f"Accuracy: {mlp.evaluate(X_test_scaled, y_test)}")

547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.7208 - loss: 0.5689
Accuracy: [0.5688197612762451, 0.7165142893791199]
